In [35]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm0104",
    security_token="EwVPSL02qXM8XQ2pkUdA13aW",
)

Descarga de pacientes y empresas

In [36]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
   select Id, Name, IdentificationId__pc, Identification_Type__c,Holding__c ,Pa_s_del_Paciente__c,Campa_a__c,AccountNumber,Id_Empresa__c,poliza__c,Empleador__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    ["Id", "Name", "IdentificationId__pc", "Identification_Type__c", "Holding__c", "Pa_s_del_Paciente__c",'Campa_a__c','AccountNumber','Id_Empresa__c','Empleador__c','poliza__c']
]
Empresas_SF.shape

In [ ]:
# recuperar todos los pacientes ID desdde aAccount
results = sf.query_all(
    """
  select Id, IdentificationId__pc,Name,poliza__c, Identification_Type__c, IsPersonAccount, Ocupaci_n__c, Pa_s_del_Paciente__c, PersonTitle ,PersonDepartment  from Account where ispersonaccount = True
    """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[["Id", "IdentificationId__pc", "Name",'poliza__c','Identification_Type__c','IsPersonAccount','Ocupaci_n__c','Pa_s_del_Paciente__c','PersonTitle','PersonDepartment']]
Pacientes_SF.shape

In [ ]:
# recuperar todos los pacientes ID desde Contact
results = sf.query_all(
    """
   select Id,Name,AccountId  from Contact
    """
)
Pacientesc_SF = pd.DataFrame(results["records"])

Pacientesc_SF = Pacientesc_SF[["Id", "AccountId", "Name"]]
Pacientesc_SF.shape

In [ ]:
#Relacion AccountContactRelacion se descarga la relacion paciente Empresa en account contact no es la q se ve
results = sf.query_all(
    """
   select Id,AccountId,ContactId,NameAccount__c, RutContact__c from AccountContactRelation
    """
)
Relacion_Pacientes_Empresas = pd.DataFrame(results["records"])

Relacion_Pacientes_Empresas = Relacion_Pacientes_Empresas[
    ["Id", "AccountId", "ContactId", "NameAccount__c", "RutContact__c"]
]
Relacion_Pacientes_Empresas.shape

In [ ]:
#union de pacientes
Union_account_contact_relation = pd.merge(
    left=Relacion_Pacientes_Empresas,
    right=Pacientesc_SF,
    how="left",
    left_on="ContactId",
    right_on="Id",
)
Union_account_contact_relation.head()

In [ ]:
#union de empresas
Union_account_contact_relation = pd.merge(
    left=Union_account_contact_relation,
    right=Empresas_SF,
    how="left",
    left_on="AccountId_x",
    right_on="Id",
)
Union_account_contact_relation.head()

In [ ]:
#union de pacientes con sus datos
Union_account_contact_relation = pd.merge(
    left=Union_account_contact_relation,
    right=Pacientes_SF,
    how="left",
    left_on="RutContact__c",
    right_on="IdentificationId__pc",
)
Union_account_contact_relation.head()

In [ ]:
#buscar empresas desde Programas_por_Empresas__c
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

In [ ]:
#buscar empresas desde Paciente_Programas__c
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF.shape

In [ ]:
#unir empresas y todas sus caracteristicas


empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Id",
)
empresas_unidas.shape

In [ ]:
empresas_unidas.head()
apprix_1 = empresas_unidas.iloc[:611178,:]
apprix_2 = empresas_unidas.iloc[611178:,:]
apprix_1.to_excel(
    "C:/Users/jmartinez/Documents/empresasA.xlsx", index=False
)
apprix_2.to_excel(
    "C:/Users/jmartinez/Documents/empresasB.xlsx", index=False
)

Descarga desde Citas de Servicios

In [ ]:
# Citas de Servicio
results = sf.query_all(
    """
    SELECT Id, clinica__c,AccountId,CreatedDate,RUT__c,Work_TypeGroup__c,Status,Interacci_n__c,AppointmentNumber,Tipo_de_la_cita__c from ServiceAppointment
    """
)
Citas_de_servicio = pd.DataFrame(results["records"])
Citas_de_servicio['Empresas'] = 'Empresas'
Citas_de_servicio['Empresas Id'] = 'Id Empresas'
Citas_de_servicio = Citas_de_servicio[
    [
        "AppointmentNumber",        
        "RUT__c",  
        'Empresas',      
        "CreatedDate",
        "Tipo_de_la_cita__c",
        'Empresas Id'        
        "AccountId",
        'Clinica__c',
        
    ]
]
Citas_de_servicio.shape


In [ ]:
# Citas de Servicio Orientacion
results = sf.query_all(
    """
    SELECT Id, AccountId,CreatedDate,RUT__c,WorkTypeId,Work_TypeGroup__c,Status,Interacci_n__c,Tipo_de_la_cita__c from ServiceAppointment
    """
)
Citas_de_servicio2 = pd.DataFrame(results["records"])
Citas_de_servicio2['Clinica__c'] = 'Orientación Médica'
Citas_de_servicio['Empresas'] = 'Empresas'
Citas_de_servicio['Empresas Id'] = 'Id Empresas'
Citas_de_servicio = Citas_de_servicio[
    [
        "AppointmentNumber",        
        "RUT__c",  
        'Empresas',      
        "CreatedDate",
        "Tipo_de_la_cita__c",
        'Empresas Id'       
        "AccountId",
        'Clinica__c',
        
    ]
]
Citas_de_servicio.shape


In [ ]:
#unir dos citas
Gran_Citas_deservicio = pd.concat([Citas_de_servicio, Citas_de_servicio2], axis=0)

In [ ]:
Gran_Citas_deservicio.shape

In [ ]:
Gran_Citas_deservicio.head()

In [ ]:
#Clinica_deporte
results = sf.query_all(
    """
   SELECT id,Name,Rut__c,Nombre_Empresa__c,CreatedDate,Canal__c, Empresa__c, Account__c from Ficha_Cl_nica_Deporte__c
     """
)
ficha_deporte = pd.DataFrame(results["records"])

ficha_deporte = ficha_deporte[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        
    ]
]
ficha_deporte['clinica'] = 'Deporte'
ficha_deporte = ficha_deporte.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",        
        'clinica':'clínica'
    }
)
ficha_deporte.shape


In [ ]:
# Clinica Nutricion
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Clinica_Nutrici_n__c 
     """
)
ficha_nutricion = pd.DataFrame(results["records"])

ficha_nutricion = ficha_nutricion[
    [
        "Name",
        "RUT__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        
    ]
]
ficha_nutricion["clinica"] = "Nutrición"

ficha_nutricion = ficha_nutricion.rename(
    columns={
        "Name": "Codigo Ficha",
        "RUT__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'clínica'
    }
)


ficha_nutricion.shape

In [ ]:
#Clinica AppSex
results = sf.query_all(
    """
   select  Name, Rut__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Cl_nica_ApsSex__c 
     """
)
ficha_Apssex = pd.DataFrame(results["records"])

ficha_Apssex = ficha_Apssex[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        
    ]
]
ficha_Apssex['clinica'] = 'Asesoría Psicologíca en Salud Sexual'
ficha_Apssex = ficha_Apssex.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'clínica'
    }
)

ficha_Apssex.shape

In [ ]:
#Clinica Psicologia
results = sf.query_all(
    """
   select  Name, Rut__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Cl_nica_Asistencia_Emocional__c 
     """
)
ficha_Psicologia = pd.DataFrame(results["records"])

ficha_Psicologia = ficha_Psicologia[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        
    ]
]
ficha_Psicologia['clinica'] = 'Psicología'
ficha_Psicologia = ficha_Psicologia.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'clínica'
    }
)
ficha_Psicologia.shape

In [ ]:
#Clinica Sueño
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Cl_nica_del_Sue_o__c 
     """
)
ficha_Sueno = pd.DataFrame(results["records"])
ficha_Sueno = ficha_Sueno[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
    ]
]
ficha_Sueno['clinica'] = 'Clínica Del Sueño'
ficha_Sueno = ficha_Sueno.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'clínica'
    }
)
ficha_Sueno.shape

In [ ]:
#Clinica PAciente Cronico
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Cl_nica_Seg_Paciente_Cr_nico__c  
     """
)
ficha_Cronico = pd.DataFrame(results["records"])
ficha_Cronico = ficha_Cronico[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
    ]
]
ficha_Cronico['clinica'] = 'Pacientes Crónicos'
ficha_Cronico = ficha_Cronico.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'clínica'
    }
)
ficha_Cronico.shape

In [ ]:
#Clinica Telemedicina
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Cl_nica_Telemedicina__c   
     """
)
ficha_Telemedicina = pd.DataFrame(results["records"])
ficha_Telemedicina = ficha_Telemedicina[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'RecordTypeId'
    ]
]
ficha_Telemedicina['clinica'] = 'Telemedicina'
ficha_Telemedicina = ficha_Telemedicina.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa Emp_pro",
        "Account__c": "Id Paciente",
        "RecordTypeId": "Tipo Ficha",
        'clinica':'clínica'
    }
)
ficha_Telemedicina.shape

In [ ]:
#Clinica Veterinaria
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Cl_nica_Veterinaria__c    
     """
)
ficha_Veterinaria = pd.DataFrame(results["records"])
ficha_Veterinaria = ficha_Veterinaria[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'RecordTypeId'
    ]
]
ficha_Veterinaria['clinica'] = 'Veterinaria'
ficha_Veterinaria = ficha_Veterinaria.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa Emp_pro",
        "Account__c": "Id Paciente",
        "RecordTypeId": "Tipo Ficha",
        'clinica':'clínica'
    }
)
ficha_Veterinaria.shape

In [ ]:
#Clinica Orientacion
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Orientaci_n_Medica__c     
     """
)
ficha_Orientacion = pd.DataFrame(results["records"])
ficha_Orientacion['RecordTypeId'] = 'Ficha de Derivacion'
ficha_Orientacion = ficha_Orientacion[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'RecordTypeId'
    ]
]
ficha_Orientacion['clinica'] = 'Orientación Médica'
ficha_Orientacion = ficha_Orientacion.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa Emp_pro",
        "Account__c": "Id Paciente",
        "RecordTypeId": "Tipo Ficha",
        'clinica':'clínica'
    }
)
ficha_Orientacion.shape

In [ ]:
#Clinica Front
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c,RecordTypeId from Ficha_Front__c     
     """
)
ficha_Front = pd.DataFrame(results["records"])

ficha_Front = ficha_Front[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'RecordTypeId'
    ]
]
ficha_Front['clinica'] = 'Orientación Médica'
ficha_Front = ficha_Front.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa Emp_pro",
        "Account__c": "Id Paciente",
        "RecordTypeId": "Tipo Ficha",
        'clinica':'clínica'
    }
)
ficha_Front.shape

In [ ]:
#Clinica Ingreso deportivo
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c,RecordTypeId from Ficha_Cl_nica_Ingreso_Deportivo__c    
     """
)
ficha_Ingreso_dep = pd.DataFrame(results["records"])

ficha_Ingreso_dep = ficha_Ingreso_dep[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'RecordTypeId'
    ]
]

ficha_Ingreso_dep = ficha_Ingreso_dep.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa Emp_pro",
        "Account__c": "Id Paciente",
        "RecordTypeId": "Tipo Ficha",
        'clinica':'clínica'
    }
)
ficha_Ingreso_dep.shape

In [ ]:
#Clinica general
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c,RecordTypeId,p from Ficha_Cl_nica_Ingreso_Deportivo__c    
     """
)
ficha_General = pd.DataFrame(results["records"])

ficha_General = ficha_General[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        
    ]
]
ficha_General['clinica'] = 'RecordTypeId'
ficha_General['clinica'] = 'Orientacion'
ficha_General = ficha_General.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa Emp_pro",
        "Account__c": "Id Paciente",
        "RecordTypeId": "Tipo Ficha",
        'clinica':'clínica'
    }
)
ficha_General.shape

In [ ]:
Clinicas = pd.concat([ficha_Front, ficha_Ingreso_dep,ficha_Orientacion,ficha_Veterinaria,ficha_Telemedicina,ficha_Cronico,ficha_Sueno,ficha_Psicologia,ficha_Apssex,ficha_nutricion,ficha_deporte], axis=0)
Clinicas.shape

In [ ]:
Clinicas.head()

In [ ]:
Gran_Citas_deservicio["Mes"] = pd.to_datetime(
    Gran_Citas_deservicio["CreatedDate"]
).dt.month

Gran_Citas_deservicio["Año"] = pd.to_datetime(
    Gran_Citas_deservicio["CreatedDate"]
).dt.year
Gran_Citas_deservicio["Dia"] = pd.to_datetime(
    Gran_Citas_deservicio["CreatedDate"]
).dt.day

# x = date(Citas_de_servicio['Dia'], Citas_de_servicio['Mes'], Citas_de_servicio['Año'] )
Gran_Citas_deservicio["Fecha"] = pd.to_datetime(
    Gran_Citas_deservicio["CreatedDate"], dayfirst=True, yearfirst=False, exact=True
)
Gran_Citas_deservicio["Fecha3"] = Gran_Citas_deservicio["Fecha"].dt.strftime("%d-%m-%Y")

clinicas_ = {
    "Clínica de Nutrición": "Nutrición",
    "Clínica de Deporte": "Deporte",
    "Clínica de Psicología": "Psicología",
    "Clínica Ingreso Deportivo Seguro": "Deporte",
    "Clínica del Sueño": "Clínica Del Sueño",
    "Clínica Seguimiento Pacientes Crónicos": "Pacientes Crónicos",
    'Orientación Médica':'Orientación Médica' ,
    'Asesoría Deportiva':'Deporte',
    'Asistencia Emocional':"Psicología",
    'Asistencia Nutricional':'Nutrición',
    'Clínica APSSex':'Asesoría Psicologíca en Salud Sexual',
    'Clínica Del Sueño':'Clínica Del Sueño',
    'Clínica Veterinaria':'Veterinaria',
    

    
}
Gran_Citas_deservicio.replace({"Clinica__c": clinicas_}, inplace=True)


Gran_Citas_deservicio.head()



In [ ]:
Gran_Citas_deservicio['Fecha'] = Gran_Citas_deservicio['Fecha'].dt.tz_localize(None)
Gran_Citas_deservicio.to_excel(
    "C:/Users/jmartinez/Documents/servicio.xlsx", index=False
)

In [ ]:
Clinicas = Clinicas.rename(
    columns={
        
        "Fecha": "CreatedDate"
        
    }
)

Clinicas["Mes"] = pd.to_datetime(
    Clinicas["CreatedDate"]
).dt.month
Clinicas["Año"] = pd.to_datetime(
    Clinicas["CreatedDate"]
).dt.year
Clinicas["Dia"] = pd.to_datetime(
    Clinicas["CreatedDate"]
).dt.day

# x = date(Citas_de_servicio['Dia'], Citas_de_servicio['Mes'], Citas_de_servicio['Año'] )
Clinicas["Fecha"] = pd.to_datetime(
    Clinicas["CreatedDate"], dayfirst=True, yearfirst=False, exact=True
)
Clinicas["Fecha3"] = Clinicas["Fecha"].dt.strftime("%d-%m-%Y")


Clinicas['Fecha'] = Clinicas['Fecha'].dt.tz_localize(None)
Clinicas.to_excel(
    "C:/Users/jmartinez/Documents/clinicas.xlsx", index=False
)
Clinicas.dtypes

